In [1]:
import numpy as np
import pandas as pd

### Este notebook auna todo el preprocesado necesario para transformar los datos de la carpeta RAW a PREPARED:

In [3]:
path_in  = "../raw/trabajo6.csv"
path_out = "../prep/prepared.snappy.parquet"

In [4]:
df = pd.read_csv(path_in)

### One Hot Encoding

In [5]:
df = df.join(pd.get_dummies(df['type'])).drop(columns=['type'])

### Nuevas variables

In [6]:
df['diffOrigen'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffDestino'] = df['newbalanceDest'] - df['oldbalanceDest']

mean_oldOrg  = df['oldbalanceOrg'].mean()
mean_oldDest = df['oldbalanceDest'].mean()

# Para evitar NaN. No afecta la distribución de los datos.
df['cambioOrigen'] = (df['diffOrigen'] + 0.01)/(df['oldbalanceOrg'] + 1e3)
df['cambioDestino'] = (df['diffDestino'] + 0.01)/(df['oldbalanceDest'] + 1e3)

### Write Prepared

In [6]:
df.to_parquet(path_out)